Index-Building:
The factors used include the following:
- planet mass (Mjup)
- planet radius (Rjup)
- planet temperature (K)
- planet density CALCULATE
- radiation intensity (W/m^2) CALCULATE
- star distance (pc)
- AND/OR HZ measurement CALCULATE
- star metallicity ?? fraction
- star mass (Msun)
- sun radius (Rsun)
- star age ?? Gy
- star effective temperature (K)
- interactions?

The index will consist of the following:
- planet mass +
planet radius +
planet temperature +
star distance (HZ) + 
radiation intensity

Weighing can be determined in 1 of 2 ways:
- use a correlation regression as done in Cobb Douglas
- use difference from "ideal" Earth on a normal distribution to determine the importance of outliers
- tweaking post-factum to ensure earth is No.1

1. Import data and packages

In [49]:
# import packages
import numpy as np
import pandas as pd
import math 
from math import pi
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)  # None means no limit
pd.set_option('display.max_rows', None) 


# import data
data = pd.read_csv('C:/Users/21sko/Desktop/dissertation/data/exo_data.csv')

#display data
# print(data)

2. Calculate density

2.1 Convert: 
- M(jup) to M(earth)
- R(jup) to R(earth)

In [50]:
# M(jup) to M(earth)
def mjup_to_mearth(planet_mass_mjup):
    mass_mearth = planet_mass_mjup * 317.8 

    return mass_mearth

data['planet_mass_mearth'] = data.apply(lambda row: mjup_to_mearth(row['planet_mass_mjup']), axis = 1)



# R(jup) to R(earth)
def rjup_to_rearth(planet_radius_rjup):
    radius_rearth = planet_radius_rjup * 11.2

    return radius_rearth

data['planet_radius_rearth'] = data.apply(lambda row: rjup_to_rearth(row['planet_radius_rjup']), axis = 1)

# print(data)

2.2 Calculate volume 

In [51]:
# 4/3 * pi* r^3
def volume_formula(planet_radius_rearth):
    volume = (4/3) * np.pi * (planet_radius_rearth ** 3)

    return volume

data['planet_volume_rearth3'] = data.apply(lambda row: volume_formula(row['planet_radius_rearth']), axis = 1)

# print(data)

2.3 Calculate relative density

In [52]:
# D = M / V

def density_formula(planet_mass_mearth, planet_volume_rearth):
    density = (planet_mass_mearth) / (planet_volume_rearth)
    
    return density


data['planet_density_relative'] = data.apply(lambda row: density_formula(row['planet_mass_mearth'], row['planet_radius_rearth']), axis = 1)

# print(data)

In [6]:
print(data)

         planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0  COCONUTS-2 b          6.400000            1.120000             434.00   
1    HAT-P-18 b          0.183000            0.947000             841.00   
2    HAT-P-42 b          0.975000            1.277000            1427.00   
3   kappa And b         13.000000            1.200000            1850.00   
4  TRAPPIST-1 b          0.002700            0.096890             451.55   
5    WASP-189 b          1.990000            1.619000            3394.00   
6         Earth          0.003146            0.089286             288.00   

     star_name  star_distance_pc  star_metallicity  star_mass_msun  \
0  COCONUTS-2A         10.890000            0.0000           0.370   
1     HAT-P-18        166.000000            0.1000           0.770   
2     HAT-P-42        447.000000            0.2700           1.179   
3    kappa And         50.000000           -0.3600           2.800   
4   TRAPPIST-1         12.100000         

2.4 Calculate absolute density = not sure if density is relevant due to a large variety of densities 

In [53]:
# 1 D(earth) = 5.52 g/cm^3

def density_relative_to_empirical(planet_density_relative):
    planet_density_empirical = planet_density_relative * 5.52
    return planet_density_empirical

data['planet_density_empirical'] = data['planet_density_relative'].apply(density_relative_to_empirical) 

print(data)

          planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0   COCONUTS-2 b      6.400000e+00            1.120000             434.00   
1     HAT-P-18 b      1.830000e-01            0.947000             841.00   
2     HAT-P-42 b      9.750000e-01            1.277000            1427.00   
3    kappa And b      1.300000e+01            1.200000            1850.00   
4   TRAPPIST-1 b      2.700000e-03            0.096890             451.55   
5     WASP-189 b      1.990000e+00            1.619000            3394.00   
6        Mercury      1.740000e-07            0.034000             440.00   
7          Venus      2.570000e-06            0.085000             737.00   
8           Mars      3.380000e-07            0.047000             210.00   
9        Jupiter      1.000000e+00            1.000000             165.00   
10        Saturn      2.990000e+00            0.815000             134.00   
11        Uranus      4.600000e-02            0.355000              76.00   

3. Calculate Habitability Zone boundaries

3.1 Version 1: simpler

In [56]:
# Function to convert mass to luminosity 
def mass_to_luminosity(star_mass_msun):
    return star_mass_msun  ** 3.6

# Function to calculate habitable zone boundaries 
def calculate_habitable_zone(Luminosity_Lsun):
    inner_boundary = np.sqrt(Luminosity_Lsun / 1.1)
    outer_boundary = np.sqrt(Luminosity_Lsun / 0.53)
    return inner_boundary, outer_boundary

def is_in_hz(lower, upper, distance):
    return lower <= distance <= upper

# Convert mass to luminosity 
data['Luminosity_Lsun'] = data['star_mass_msun'].apply(mass_to_luminosity) 

# Calculate HZ boundaries and add them to the dataframe 
data[['HZ_inner_AU_s', 'HZ_outer_AU_s']] = data['Luminosity_Lsun'].apply( lambda luminosity: calculate_habitable_zone(luminosity) ).apply(pd.Series)


data['Is_In_HZ'] = data.apply(lambda row: is_in_hz(row['HZ_inner_AU_s'], row['HZ_outer_AU_s'], row['star_distance_AU']), axis=1)

# Display the updated dataset to the user in a Jupyter-friendly format 
print(data)

KeyError: 'star_distance_AU'

3.2 Version 2: More complex

In [55]:
# Function to convert mass to luminosity 
def mass_to_luminosity(star_mass_msun):
    return star_mass_msun  ** 3.6

# Function to calculate habitable zone boundaries 
def calculate_habitable_zone(Luminosity_Lsun):
    inner_boundary = np.sqrt(Luminosity_Lsun / 1.1)
    outer_boundary = np.sqrt(Luminosity_Lsun / 0.53)
    return inner_boundary, outer_boundary 

# Convert mass to luminosity 
data['Luminosity_Lsun'] = data['star_mass_msun'].apply(mass_to_luminosity) 



# inner flux 
def s_inner(Teff):
    s_inner = 1.296 - 2.139e-4 * Teff + 4.19e-8 * (Teff)**2

    return s_inner

data['Flux_inner_HZ'] = data['star_teff_K'].apply(s_inner) 

# outer flux
def s_outer(Teff):
    s_outer = 0.234 - 1.319e-5 * Teff + 6.19e-10 * (Teff)**2

    return s_outer

data['Flux_outer_HZ'] = data['star_teff_K'].apply(s_outer) 

# inner boundary
def r_inner(Luminosity_Lsun, Flux_inner_HZ):
    r_inner = np.sqrt(Luminosity_Lsun / Flux_inner_HZ)

    return r_inner

data['HZ_inner_AU_2'] = data.apply(lambda row: r_inner(row['Luminosity_Lsun'], row['Flux_inner_HZ']), axis = 1)


# outer boundary
def r_outer(Luminosity_Lsun, Flux_outer_HZ):
    r_outer = np.sqrt(Luminosity_Lsun / Flux_outer_HZ)

    return r_outer

data['HZ_outer_AU_2'] = data.apply(lambda row: r_outer(row['Luminosity_Lsun'], row['Flux_outer_HZ']), axis = 1)

# print(data)

4. Calculate middle of HZ

In [57]:
def HZ_middle_ideal(HZ_inner, HZ_outer):
    HZ_middle = (HZ_inner + HZ_outer) / 2

    return HZ_middle

data['HZ_middle_AU_s'] = data.apply(lambda row: HZ_middle_ideal(row['HZ_inner_AU_s'], row['HZ_outer_AU_s']), axis = 1)
data['HZ_middle_ideal_AU'] = data.apply(lambda row: HZ_middle_ideal(row['HZ_inner_AU_2'], row['HZ_outer_AU_2']), axis = 1)


5. Radiation Intensity
- Calculate distance from planet to star in m

In [58]:
def pc_to_m(star_distance_pc):
    star_distance_m = star_distance_pc * 3.086e+16

    return star_distance_m

data['star_distance_m'] = data['star_distance_pc'].apply(pc_to_m) 

# print(data)


In [86]:
# radiation = L / 4pi(r)^2 W/m^2
def radiation_I(luminosity, distance_m):
    radiation_I = luminosity / (4*np.pi*((distance_m)**2))

    return radiation_I


data['radiation_I'] = data.apply(lambda row: radiation_I(row['Luminosity_Lsun'], row['star_distance_m']), axis = 1)

In [84]:
print(data)



          planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0   COCONUTS-2 b      6.400000e+00            1.120000             434.00   
1     HAT-P-18 b      1.830000e-01            0.947000             841.00   
2     HAT-P-42 b      9.750000e-01            1.277000            1427.00   
3    kappa And b      1.300000e+01            1.200000            1850.00   
4   TRAPPIST-1 b      2.700000e-03            0.096890             451.55   
5     WASP-189 b      1.990000e+00            1.619000            3394.00   
6        Mercury      1.740000e-07            0.034000             440.00   
7          Venus      2.570000e-06            0.085000             737.00   
8           Mars      3.380000e-07            0.047000             210.00   
9        Jupiter      1.000000e+00            1.000000             165.00   
10        Saturn      2.990000e+00            0.815000             134.00   
11        Uranus      4.600000e-02            0.355000              76.00   

5. Calculate pre-main sequence time and main-sequence lifetime for given stars. ideal = middle main sequence, non-ideal = pre or post-main sequence

5.1 Calculate pre-main sequence phase (tPMS)

In [61]:
def pre_seq_GY(star_mass_msun):
    pre_seq_length = ( 10e7*(star_mass_msun**-2.5) ) / 10e9

    return pre_seq_length

data['pre_seq_GY'] = data['star_mass_msun'].apply(pre_seq_GY) 
# print(data)

5.2 Calculate main-sequence lifetime (tMS)

In [62]:
def main_seq_GY(star_mass_msun):
    main_seq_length = 10*(1/(star_mass_msun**2.5))

    return main_seq_length

data['main_seq_GY'] = data['star_mass_msun'].apply(main_seq_GY) 

# print(data)

5.3 Calculate end of main sequence phase (transition into post-main sequence)

In [63]:
data['main_seq_end_GY'] = data['pre_seq_GY'] + data['main_seq_GY']

# print(data)

5.4 Calculate mid-main sequence time

In [64]:
def mid_main_seq_GY(main_seq_end_GY, pre_seq_GY):
    mid_main_seq = (main_seq_end_GY - pre_seq_GY) / 2

    return mid_main_seq

data['mid_main_seq_GY'] = data.apply(lambda row: mid_main_seq_GY(row['main_seq_end_GY'], row['pre_seq_GY']), axis = 1)
# print(data)

In [65]:
print(data)

          planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0   COCONUTS-2 b      6.400000e+00            1.120000             434.00   
1     HAT-P-18 b      1.830000e-01            0.947000             841.00   
2     HAT-P-42 b      9.750000e-01            1.277000            1427.00   
3    kappa And b      1.300000e+01            1.200000            1850.00   
4   TRAPPIST-1 b      2.700000e-03            0.096890             451.55   
5     WASP-189 b      1.990000e+00            1.619000            3394.00   
6        Mercury      1.740000e-07            0.034000             440.00   
7          Venus      2.570000e-06            0.085000             737.00   
8           Mars      3.380000e-07            0.047000             210.00   
9        Jupiter      1.000000e+00            1.000000             165.00   
10        Saturn      2.990000e+00            0.815000             134.00   
11        Uranus      4.600000e-02            0.355000              76.00   

6. Calculate water potential !!!!

In [66]:
def pc_to_AU(star_distance_pc):
    star_distance_AU = star_distance_pc  * 206265

    return star_distance_AU

data['star_distance_AU'] = data['star_distance_pc'].apply(pc_to_AU) 
print(data)

          planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0   COCONUTS-2 b      6.400000e+00            1.120000             434.00   
1     HAT-P-18 b      1.830000e-01            0.947000             841.00   
2     HAT-P-42 b      9.750000e-01            1.277000            1427.00   
3    kappa And b      1.300000e+01            1.200000            1850.00   
4   TRAPPIST-1 b      2.700000e-03            0.096890             451.55   
5     WASP-189 b      1.990000e+00            1.619000            3394.00   
6        Mercury      1.740000e-07            0.034000             440.00   
7          Venus      2.570000e-06            0.085000             737.00   
8           Mars      3.380000e-07            0.047000             210.00   
9        Jupiter      1.000000e+00            1.000000             165.00   
10        Saturn      2.990000e+00            0.815000             134.00   
11        Uranus      4.600000e-02            0.355000              76.00   

In [67]:
import math

# Constants
k_gravity = 6.67430e-11  # Gravitational constant, m^3 kg^-1 s^-1
k_Boltzman = 1.380649e-23  # Boltzmann constant, J K^-1
AU = 1.496e11  # Astronomical unit in meters
molecular_mass=4.65e-26 # N2
earth_mass=5.972e24
earth_radius=6.371e6
L_sun = 3.828e26 

# Functions to calculate necessary parameters
"""
Calculate the surface gravity of the planet.
mass: Mass of the planet in kg == Mearth?
radius: Radius of the planet in meters == Rearth
"""
def calculate_surface_gravity(mass, radius):
    return k_gravity * (mass*earth_mass) / (radius*earth_radius)**2



"""
Calculate the escape velocity of the planet.
mass: Mass of the planet in kg
radius: Radius of the planet in meters
"""
def calculate_escape_velocity(mass, radius):
   
    return math.sqrt(2 * k_gravity * (mass*earth_mass) / (radius*earth_radius))





"""
Calculate the thermal velocity of atmospheric molecules.
temperature: Surface temperature in Kelvin
molecular_mass: Mass of the gas molecule in kg (for N2, approximately 4.65e-26 kg)
"""
def calculate_thermal_velocity(temperature, molecular_mass):
    return math.sqrt(3 * k_Boltzman * temperature / molecular_mass)





"""
Determine if the planet is within the habitable zone of its star.
distance: Distance from the star in AU
luminosity: Luminosity of the star in watts
"""
def is_in_habitable_zone(distance, luminosity):
    inner_boundary = math.sqrt((luminosity) / (1.1))
    outer_boundary = math.sqrt((luminosity) / (0.53))
    return inner_boundary <= distance <= outer_boundary


"""
Evaluate the probability of the planet being able to sustain liquid water.
mass: Mass of the planet in kg
radius: Radius of the planet in meters
distance: Distance from the star in AU
temperature: Surface temperature in Kelvin
luminosity: Luminosity of the star in watts (default is Sun-like)
molecular_mass: Mass of the atmospheric gas molecule in kg (default is for nitrogen, N2)
"""
def evaluate_water_sustenance(mass, radius, distance, temperature, luminosity, molecular_mass= 4.65e-26):
    gravity = calculate_surface_gravity(mass, radius)
    escape_velocity = calculate_escape_velocity(mass, radius)
    thermal_velocity = calculate_thermal_velocity(temperature, molecular_mass)
    in_habitable_zone = is_in_habitable_zone(distance, luminosity)
    
    # Surface gravity assessment (reasonable range 0.38g to 2g where g is Earth's gravity)
    gravity_assessment = (0.38 * 9.81) <= gravity <= (2 * 9.81)
    
    # Atmospheric retention assessment (escape velocity should be much higher than thermal velocity)
    atmospheric_retention = 0 if thermal_velocity == 0 else escape_velocity > 10 * thermal_velocity
    
    # Surface temperature assessment (within 0°C to 100°C for liquid water)
    temperature_assessment = 273 <= temperature <= 373
    
    # Combine assessments into a probability category
    """
    if in_habitable_zone and temperature_assessment and atmospheric_retention and gravity_assessment:
        return 0.75
    elif in_habitable_zone and (temperature_assessment or atmospheric_retention or gravity_assessment):
        return 0.5
    else:
        return 0.25
    """
    return (in_habitable_zone + temperature_assessment + atmospheric_retention + gravity_assessment) * 0.25

# Example use case:
# data['mid_main_seq_GY'] = data.apply(lambda row: mid_main_seq_GY(row['main_seq_end_GY'], row['pre_seq_GY']), axis = 1)


data["water"] = data.apply(lambda row: evaluate_water_sustenance(row['planet_mass_mearth'], row['planet_radius_rearth'], row['star_distance_AU'], row['temp_calculated_K'],row['Luminosity_Lsun']), axis = 1)
    #evaluate_water_sustenance(planet_mass, planet_radius, distance_from_star, surface_temperature)
print(data)


          planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0   COCONUTS-2 b      6.400000e+00            1.120000             434.00   
1     HAT-P-18 b      1.830000e-01            0.947000             841.00   
2     HAT-P-42 b      9.750000e-01            1.277000            1427.00   
3    kappa And b      1.300000e+01            1.200000            1850.00   
4   TRAPPIST-1 b      2.700000e-03            0.096890             451.55   
5     WASP-189 b      1.990000e+00            1.619000            3394.00   
6        Mercury      1.740000e-07            0.034000             440.00   
7          Venus      2.570000e-06            0.085000             737.00   
8           Mars      3.380000e-07            0.047000             210.00   
9        Jupiter      1.000000e+00            1.000000             165.00   
10        Saturn      2.990000e+00            0.815000             134.00   
11        Uranus      4.600000e-02            0.355000              76.00   

6. Calculate ideal and non-ideal values ??

- planet mass M(earth) == Earth? between 0.5 and 5 earth masses as can maintain atmosphere and possibly have H2O (aq)
- planet radius R(earth) == Earth? between 0.8 and 1.5 earth radii as above might have thick gas envelopes and be more like mini-neptures/gas giant
- planet temperature (K) == Earth? between 0 and 100'c where water can remain liquid as standard pressure
- planet density == Earth? between 5 and 5.5 g/cm3
- radiation intensity (W/m^2) == Earth ? limits at HZ?
- HZ measurement == normal distribution, y = middle of HZ; non ideal = | 2sigma | where 1.5 sigma == boundary 
- star age ?? Gy maybe calculate lifetime based on Tms = 10gy * (1/(Mstar^2.5))  where Tms is main sequence lifetime. Pre-main sequence phase tpms = 10^7 * M^-2.5
- interactions??

In [21]:
print(data)

         planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0  COCONUTS-2 b          6.400000            1.120000             434.00   
1    HAT-P-18 b          0.183000            0.947000             841.00   
2    HAT-P-42 b          0.975000            1.277000            1427.00   
3   kappa And b         13.000000            1.200000            1850.00   
4  TRAPPIST-1 b          0.002700            0.096890             451.55   
5    WASP-189 b          1.990000            1.619000            3394.00   
6         Earth          0.003146            0.089286             288.00   

     star_name  star_distance_pc  star_metallicity  star_mass_msun  \
0  COCONUTS-2A         10.890000            0.0000           0.370   
1     HAT-P-18        166.000000            0.1000           0.770   
2     HAT-P-42        447.000000            0.2700           1.179   
3    kappa And         50.000000           -0.3600           2.800   
4   TRAPPIST-1         12.100000         

6.1 Mass ideal and non-ideal values -> in water index?

In [22]:
# ?????

6.2 Radius ideal and non-ideal values

In [23]:
# ?????
print(data)

         planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0  COCONUTS-2 b          6.400000            1.120000             434.00   
1    HAT-P-18 b          0.183000            0.947000             841.00   
2    HAT-P-42 b          0.975000            1.277000            1427.00   
3   kappa And b         13.000000            1.200000            1850.00   
4  TRAPPIST-1 b          0.002700            0.096890             451.55   
5    WASP-189 b          1.990000            1.619000            3394.00   
6         Earth          0.003146            0.089286             288.00   

     star_name  star_distance_pc  star_metallicity  star_mass_msun  \
0  COCONUTS-2A         10.890000            0.0000           0.370   
1     HAT-P-18        166.000000            0.1000           0.770   
2     HAT-P-42        447.000000            0.2700           1.179   
3    kappa And         50.000000           -0.3600           2.800   
4   TRAPPIST-1         12.100000         

6.3 Temperature ideal and non-ideal values

In [68]:
# calculate temperature in C

def K_to_C(temp_calculated_K):

    temp_calculated_C = temp_calculated_K - 273

    return temp_calculated_C
data['temp_calculated_C'] = data['temp_calculated_K'].apply(K_to_C) 


## ideal = 20 X
## non ideal: 0 and 100

In [71]:
# we need a buffer....
# i timesed them by 2. check relevance

In [74]:
def map_temperature(value, V_ideal=15, V_min=-20*2, V_max=100*2, d=1):
   
    # Calculate distances from ideal to min and max
    distance_min = V_ideal - V_min
    distance_max = V_max - V_ideal
    
    # Determine the position of the value relative to V_ideal and normalize to [0, d]
    if value <= V_ideal:
        # Map values between V_min and V_ideal to [0, d]
        new_value = d * (V_ideal - value) / distance_min
    else:
        # Map values between V_ideal and V_max to [0, d]
        new_value = d * (value - V_ideal) / distance_max
    
    new_value = new_value if new_value <=1 else 1
    return d/2 + (d/2 - new_value)



data['map_temperature'] = data['temp_calculated_C'].apply(map_temperature) 
print(data)

          planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0   COCONUTS-2 b      6.400000e+00            1.120000             434.00   
1     HAT-P-18 b      1.830000e-01            0.947000             841.00   
2     HAT-P-42 b      9.750000e-01            1.277000            1427.00   
3    kappa And b      1.300000e+01            1.200000            1850.00   
4   TRAPPIST-1 b      2.700000e-03            0.096890             451.55   
5     WASP-189 b      1.990000e+00            1.619000            3394.00   
6        Mercury      1.740000e-07            0.034000             440.00   
7          Venus      2.570000e-06            0.085000             737.00   
8           Mars      3.380000e-07            0.047000             210.00   
9        Jupiter      1.000000e+00            1.000000             165.00   
10        Saturn      2.990000e+00            0.815000             134.00   
11        Uranus      4.600000e-02            0.355000              76.00   

6.4 Radiation intensity
!!!!!!!!! this needs work

In [26]:
# ideal = intensity at LB
# non-ideal = intensity at sun

# print(data)

In [72]:
print(data)

          planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0   COCONUTS-2 b      6.400000e+00            1.120000             434.00   
1     HAT-P-18 b      1.830000e-01            0.947000             841.00   
2     HAT-P-42 b      9.750000e-01            1.277000            1427.00   
3    kappa And b      1.300000e+01            1.200000            1850.00   
4   TRAPPIST-1 b      2.700000e-03            0.096890             451.55   
5     WASP-189 b      1.990000e+00            1.619000            3394.00   
6        Mercury      1.740000e-07            0.034000             440.00   
7          Venus      2.570000e-06            0.085000             737.00   
8           Mars      3.380000e-07            0.047000             210.00   
9        Jupiter      1.000000e+00            1.000000             165.00   
10        Saturn      2.990000e+00            0.815000             134.00   
11        Uranus      4.600000e-02            0.355000              76.00   

In [90]:
# radiation = L / 4pi(r)^2 W/m^2
# 1**2 means 1m away from the sun
data['HZ_inner_m_s'] = data['HZ_inner_AU_s'] * 149597870.7
data['HZ_inner_m_2'] = data['HZ_inner_AU_2'] * 149597870.7
data['HZ_outer_m_s'] = data['HZ_outer_AU_s'] * 149597870.7
data['HZ_outer_m_2'] = data['HZ_outer_AU_2'] * 149597870.7




def radiation_I_max(luminosity):
    radiation_max_I = luminosity / (4*np.pi*((1)**2))
    return radiation_max_I
data['radiation_max'] = data.apply(lambda row: radiation_I_max(row['Luminosity_Lsun']), axis = 1)




def radiation_I_ideal(luminosity, distance_m):
    radiation_ideal_I = luminosity / (4*np.pi*((distance_m)**2))

    return radiation_ideal_I
data['radiation_min'] = data.apply(lambda row: radiation_I_ideal(row['Luminosity_Lsun'], row['HZ_outer_m_2']), axis = 1)

print(data)

          planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0   COCONUTS-2 b      6.400000e+00            1.120000             434.00   
1     HAT-P-18 b      1.830000e-01            0.947000             841.00   
2     HAT-P-42 b      9.750000e-01            1.277000            1427.00   
3    kappa And b      1.300000e+01            1.200000            1850.00   
4   TRAPPIST-1 b      2.700000e-03            0.096890             451.55   
5     WASP-189 b      1.990000e+00            1.619000            3394.00   
6        Mercury      1.740000e-07            0.034000             440.00   
7          Venus      2.570000e-06            0.085000             737.00   
8           Mars      3.380000e-07            0.047000             210.00   
9        Jupiter      1.000000e+00            1.000000             165.00   
10        Saturn      2.990000e+00            0.815000             134.00   
11        Uranus      4.600000e-02            0.355000              76.00   

In [93]:
def map_radiation(value, radiation_min, radiation_max, d=1):
   
    # Calculate distances from ideal to min and max
    distance = abs(radiation_max - radiation_min)
    
    # Determine the position of the value relative to V_ideal and normalize to [0, d]
    if value <= radiation_min:
        # Map values between V_min and V_ideal to [0, d]
        new_value = d * (radiation_min - value) / distance
    else:
        # Map values between V_ideal and V_max to [0, d]
        new_value = d * (radiation_min - radiation_min) / distance
    
    # new_value = new_value if new_value <=1 else 1
    return d/2 + (d/2 - new_value)



data['map_radiation'] = data.apply(lambda row: map_radiation(row['radiation_I'], row['radiation_min'], row['radiation_max']), axis = 1)
print(data)

SyntaxError: Function parameters cannot be parenthesized (4038451026.py, line 1)

In [94]:
print(data)

          planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0   COCONUTS-2 b      6.400000e+00            1.120000             434.00   
1     HAT-P-18 b      1.830000e-01            0.947000             841.00   
2     HAT-P-42 b      9.750000e-01            1.277000            1427.00   
3    kappa And b      1.300000e+01            1.200000            1850.00   
4   TRAPPIST-1 b      2.700000e-03            0.096890             451.55   
5     WASP-189 b      1.990000e+00            1.619000            3394.00   
6        Mercury      1.740000e-07            0.034000             440.00   
7          Venus      2.570000e-06            0.085000             737.00   
8           Mars      3.380000e-07            0.047000             210.00   
9        Jupiter      1.000000e+00            1.000000             165.00   
10        Saturn      2.990000e+00            0.815000             134.00   
11        Uranus      4.600000e-02            0.355000              76.00   

6.5 HZ normal distribution; ideal and non-ideal values

In [95]:

def map_HZ(value, mid_ideal, min_nonideal, max_nonideal, d=1):
   
    # Calculate distances from ideal to min and max
    distance_min = mid_ideal - (min_nonideal / 1.5 *2)
    distance_max = mid_ideal - (max_nonideal / 1.5 *2)
    
    # Determine the position of the value relative to V_ideal and normalize to [0, d]
    if value <= mid_ideal:
        # Map values between V_min and V_ideal to [0, d]
        new_value = d * (mid_ideal - value) / distance_min
    else:
        # Map values between V_ideal and V_max to [0, d]
        new_value = d * (value - mid_ideal) / distance_max
    
    #new_value = new_value if new_value <=1 else 1
    return d/2 + (d/2 - new_value)



data['map_HZ'] = data.apply(lambda row: map_HZ(row['star_distance_AU'], row['HZ_middle_ideal_AU'], row['HZ_inner_AU_2'], row['HZ_outer_AU_2']), axis = 1)
print(data)

          planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0   COCONUTS-2 b      6.400000e+00            1.120000             434.00   
1     HAT-P-18 b      1.830000e-01            0.947000             841.00   
2     HAT-P-42 b      9.750000e-01            1.277000            1427.00   
3    kappa And b      1.300000e+01            1.200000            1850.00   
4   TRAPPIST-1 b      2.700000e-03            0.096890             451.55   
5     WASP-189 b      1.990000e+00            1.619000            3394.00   
6        Mercury      1.740000e-07            0.034000             440.00   
7          Venus      2.570000e-06            0.085000             737.00   
8           Mars      3.380000e-07            0.047000             210.00   
9        Jupiter      1.000000e+00            1.000000             165.00   
10        Saturn      2.990000e+00            0.815000             134.00   
11        Uranus      4.600000e-02            0.355000              76.00   

6.6 Star age ideal and non-ideal values

In [ ]:
print(data)

In [ ]:
# ideal = mid_main_seq_GY  
# non ideal: > main_seq_end_GY  OR < pre_seq_GY

"""
I love Sashka - Milashka
"""

7. Normalise decision matrix ?

8. Weighing
Weighing can be determined in 1 of 2 ways:
- use a correlation regression as done in Cobb Douglas
- use difference from "ideal" Earth on a normal distribution to determine the importance of outliers

9. Seperation measures - distance to ideal and non-ideal
- likely to include creation of a completely new dataset for comparison of EACH exoplanet

10. Relative closeness to ideal

11. Comparison to Earth etc
Would be good to add a visual aspect:
- maybe an image with 'sliders' showing where points are from ideal?
-  

Future directions:
- percentile on normal